In [11]:
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed(42)

num_samples = 100  

X = np.array([
    [np.random.randint(115, 130), np.random.randint(4, 7), np.random.randint(25, 35), np.random.randint(3, 7)] for _ in range(num_samples)  # Bouncer
] + [
    [np.random.randint(125, 140), np.random.randint(5, 8), np.random.randint(5, 15), np.random.randint(12, 18)] for _ in range(num_samples)  # Inswinger
] + [
    [np.random.randint(120, 135), np.random.randint(4, 7), np.random.randint(10, 20), np.random.randint(-12, -6)] for _ in range(num_samples)  # Reverse swing
] + [
    [np.random.randint(105, 120), np.random.randint(6, 9), np.random.randint(15, 25), np.random.randint(6, 12)] for _ in range(num_samples)  # Leg cutter
] + [
    [np.random.randint(100, 115), np.random.randint(5, 8), np.random.randint(20, 30), np.random.randint(-7, -2)] for _ in range(num_samples)  # Off cutter
] + [
    [np.random.randint(120, 135), np.random.randint(4, 6), np.random.randint(8, 18), np.random.randint(15, 20)] for _ in range(num_samples)  # Outswinger
] + [
    [np.random.randint(140, 155), np.random.randint(3, 5), np.random.randint(3, 8), np.random.randint(0, 4)] for _ in range(num_samples)  # Yorker
] + [
    [np.random.randint(95, 110), np.random.randint(7, 10), np.random.randint(15, 22), np.random.randint(-3, 3)] for _ in range(num_samples)  # Knuckleball
] + [
    [np.random.randint(85, 100), np.random.randint(8, 12), np.random.randint(18, 25), np.random.randint(2, 6)] for _ in range(num_samples)  # Slower ball
])


y = (["Bouncer"] * num_samples + ["Inswinger"] * num_samples + ["Reverse swing"] * num_samples +
     ["Leg cutter"] * num_samples + ["Off cutter"] * num_samples + ["Outswinger"] * num_samples +
     ["Yorker"] * num_samples + ["Knuckleball"] * num_samples + ["Slower ball"] * num_samples)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

with open("ball_type_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Improved model saved as 'ball_type_model.pkl'")


Model Accuracy: 98.89%
Improved model saved as 'ball_type_model.pkl'


In [3]:
import cv2
import numpy as np
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
import pickle

with open("ball_type_model.pkl", "rb") as f:
    model = pickle.load(f)

def extract_ball_features(traj_x, traj_y, speeds):
    """
    Extracts features like bounce height, speed, deviation, and spin for classification.
    """
    if len(traj_x) < 2:
        return None

    avg_speed = np.mean(speeds)
    speed_variation = np.std(speeds)


    bounce_y = min(traj_y) 
    bounce_index = traj_y.index(bounce_y)


    if bounce_index > 0:
        deviation_angle = np.arctan2(traj_y[-1] - traj_y[bounce_index], traj_x[-1] - traj_x[bounce_index]) * (180 / np.pi)
    else:
        deviation_angle = 0  

    return [avg_speed, speed_variation, bounce_y, deviation_angle]

def classify_ball_type(video_path):
    """
    Process video to classify ball type based on extracted features.
    """
    cap = cv2.VideoCapture(video_path)
    
   
    traj_x, traj_y = [], []
    speeds = []
    prev_x, prev_y = None, None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                   param1=50, param2=30, minRadius=5, maxRadius=15)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            x, y, r = circles[0][0]

        
            traj_x.append(x)
            traj_y.append(y)

     
            if prev_x is not None:
                x, y, prev_x, prev_y = map(float, [x, y, prev_x, prev_y])  
                speed = np.sqrt((x - prev_x) ** 2 + (y - prev_y) ** 2)
                speeds.append(speed)

            prev_x, prev_y = x, y

   
        cv2.imshow('Ball Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

 
    features = extract_ball_features(traj_x, traj_y, speeds)
    if features:
        prediction = model.predict([features])[0]
        print(f"Predicted Ball Type: {prediction}")
    else:
        print("Ball tracking failed. Try another video.")


video_path = "C:\\Users\\chand\\Downloads\\SlowMoCric_5.mp4"
classify_ball_type(video_path)


Predicted Ball Type: Yorker
